<a href="https://colab.research.google.com/github/Srajan1122/Hyperspectral_Image_Classification/blob/srajan/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from scipy import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing
import keras

Using TensorFlow backend.


# Importing Dataset

In [0]:
loaded_dataset = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/PaviaU/PaviaU.mat')
for key, value in loaded_dataset.items():
  if isinstance(value, type(np.array([1]))):
    image = loaded_dataset[key]

In [0]:
ground_truth_1 = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/PaviaU/PaviaU_gt.mat')
for key, value in ground_truth_1.items():
  if isinstance(value, type(np.array([1]))):
    ground_truth = ground_truth_1[key]

## Class Labels

In [0]:
label_values = ["Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]

## Resizing the data

In [0]:
image_with_gt = np.dstack((image, ground_truth))

In [0]:
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)

## Data Visualization in pandas

In [0]:
data = pd.DataFrame(final_output)

In [9]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
0,647,499,464,371,291,319,365,322,296,305,277,219,219,222,201,162,157,183,204,194,198,216,249,284,294,322,338,342,336,342,362,365,348,341,324,316,293,274,251,244,...,192,227,287,376,493,633,763,913,1149,1442,1759,2102,2425,2689,2895,3058,3196,3252,3195,3297,3542,3550,3537,3545,3514,3477,3468,3433,3408,3420,3416,3335,3256,3226,3205,3210,3221,3238,3250,0
1,604,546,527,455,378,377,336,314,324,329,328,326,275,215,197,217,239,258,246,233,241,261,279,300,329,344,341,362,362,367,368,359,345,344,327,302,281,271,270,261,...,152,190,242,301,391,505,633,774,960,1191,1454,1737,2001,2209,2363,2484,2594,2613,2541,2588,2821,2839,2738,2690,2678,2673,2712,2725,2657,2605,2583,2538,2509,2511,2501,2468,2442,2464,2528,0
2,621,746,556,360,285,300,274,276,332,319,278,255,232,207,210,209,168,139,137,154,168,202,225,231,244,257,263,262,266,276,285,291,285,272,271,241,208,194,179,157,...,116,153,188,243,322,413,492,591,737,933,1125,1327,1519,1666,1790,1934,2029,2034,2010,2069,2191,2233,2245,2249,2255,2250,2249,2239,2239,2282,2288,2242,2193,2202,2225,2266,2308,2345,2361,0
3,637,592,482,556,508,284,178,198,193,224,217,215,187,161,152,136,124,122,127,126,125,123,147,188,208,203,202,206,219,235,238,227,217,215,216,201,190,167,149,162,...,129,174,245,338,455,585,720,850,1013,1225,1460,1702,1938,2157,2317,2433,2525,2552,2493,2554,2733,2781,2748,2748,2737,2710,2681,2642,2645,2684,2675,2609,2541,2551,2571,2569,2573,2620,2644,0
4,625,560,480,360,377,341,264,208,231,266,207,154,123,95,106,102,101,123,154,170,191,210,216,256,289,325,348,357,361,362,363,370,386,369,344,331,312,308,288,263,...,173,220,292,382,488,592,692,834,1032,1236,1461,1698,1876,2007,2118,2213,2288,2267,2192,2274,2448,2454,2387,2388,2441,2435,2454,2457,2430,2423,2392,2344,2289,2328,2415,2424,2386,2431,2456,0


## Droping the rows if ground truth value is zero

In [10]:
print("Percentage of column which will be droped",(data.size - data[data.iloc[:, -1] == 0].size)/data.size,"%")

Percentage of column which will be droped 0.20624879459980713 %


In [0]:
zero_data = data.index[data.iloc[:, -1] == 0].tolist()
data = data[data.iloc[:, -1] != 0]

# Spliting the data into feature and target

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:,-1]

In [0]:
number_of_classes = len(np.unique(y))

# Feature Selection

In [0]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

# X = SelectKBest(f_classif, k=int((image.shape[2]+1)*0.75)).fit_transform(X, y)

# Feature Extraction 


In [0]:
from sklearn.decomposition import PCA 

pca = PCA(n_components = int((image.shape[2]+1)*0.75))
X = pca.fit_transform(X)

# OneHotEncoding in target column

In [0]:
onehotencoder = OneHotEncoder() 
y = onehotencoder.fit_transform(np.array(y).reshape(-1,1)).toarray() 

# Standardizing the feature columna

In [0]:
X = preprocessing.scale(X)

# Spliting the data into training and testing set 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60, random_state=1)#0.25 0.15

In [0]:
number = int(X_test.shape[0]/2)

xx_test = X_test[:number, :]
xx_val = X_test[number:, :]

if len(y_test.shape) > 1:
  loss = keras.losses.categorical_crossentropy
  metrics=['accuracy']
  yy_test = y_test[:number, :]
  yy_val = y_test[number:, :]
else:
  loss = keras.losses.sparse_categorical_crossentropy
  metrics=['sparse_categorical_accuracy']
  y_test = [x - 1 for x in y_test]
  y_train = [x - 1 for x in y_train]
  yy_test = y_test[:number]
  yy_val = y_test[number:]

In [0]:
from keras import backend as K
feature = X.shape[1]

if K.image_data_format() == 'channels_first':
    X = X.reshape(X.shape[0], 1, feature)
    X_train = X_train.reshape(X_train.shape[0], 1, feature)
    xx_test = xx_test.reshape(xx_test.shape[0], 1, feature)
    xx_val = xx_val.reshape(xx_val.shape[0], 1, feature)
    input_shape = (1, feature)
else:
    X = X.reshape(X.shape[0], feature, 1)
    X_train = X_train.reshape(X_train.shape[0], feature, 1)
    xx_test = xx_test.reshape(xx_test.shape[0], feature, 1)
    xx_val = xx_val.reshape(xx_val.shape[0], feature, 1)
    input_shape = (feature, 1)

# Genetic Algorithm

In [21]:
!pip install pyeasyga

  Created wheel for pyeasyga: filename=pyeasyga-0.3.1-py2.py3-none-any.whl size=6774 sha256=ee41cfc519825d25a74abe11e8b95972b26f30f7870d56709dd8c917b34ed085
  Stored in directory: /root/.cache/pip/wheels/ef/cf/ef/7aff9fcd6c1e59dc276182f29a32e7c197665dd5eb547f30e6
Successfully built pyeasyga


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization

In [0]:
# from pyeasyga import pyeasyga
# import random
# import numpy as np
# from sklearn.metrics import accuracy_score
# import keras
# import time

# count = 1

# def initilialize_population():
#     filter_1 = list(x for x in range(1, 11))
#     filter_2 = list(x for x in range(11, 31))
#     filter_3 = list(x for x in range(31, 61))
#     filter_4 = list(x for x in range(61, 101))

#     kernel_size_1 = [2,3]
#     kernel_size_2 = [2,3]
#     kernel_size_3 = [2,3]
#     kernel_size_4 = [2,3]


#     population = []
#     population.append(filter_1)
#     population.append(filter_2)
#     population.append(filter_3)
#     population.append(filter_4)
#     population.append(kernel_size_1)
#     population.append(kernel_size_2)
#     population.append(kernel_size_3)
#     population.append(kernel_size_4)

#     return list(population)

# def create_individual(data):
#   choice = []
#   for i in range(len(data)):
#     choice.append(random.choice(list(range(len(data[i])))))
#   return choice

# def fitness_function(individual, data):
#   global count
#   print('Individual no', count)
#   count += 1
#   model = Sequential()
#   model.add(Conv1D(filters=data[0][individual[0]], kernel_size=data[4][individual[4]], activation='relu', input_shape=input_shape))
#   model.add(Conv1D(filters=data[0][individual[0]], kernel_size=data[4][individual[4]], activation='relu', input_shape=input_shape))
#   model.add(BatchNormalization())
#   model.add(MaxPooling1D(pool_size=2))
#   model.add(Conv1D(filters=data[1][individual[1]], kernel_size=data[5][individual[5]], activation='relu'))
#   model.add(Conv1D(filters=data[1][individual[1]], kernel_size=data[5][individual[5]], activation='relu'))
#   model.add(BatchNormalization())
#   model.add(MaxPooling1D(pool_size=2))
#   model.add(Conv1D(filters=data[2][individual[2]], kernel_size=data[6][individual[6]], activation='relu'))
#   model.add(Conv1D(filters=data[2][individual[2]], kernel_size=data[6][individual[6]], activation='relu'))
#   model.add(BatchNormalization())
#   model.add(MaxPooling1D(pool_size=2))
#   model.add(Conv1D(filters=data[3][individual[3]], kernel_size=data[7][individual[7]], activation='relu'))
#   model.add(Conv1D(filters=data[3][individual[3]], kernel_size=data[7][individual[7]], activation='relu'))
#   model.add(BatchNormalization())
#   model.add(MaxPooling1D(pool_size=2))
#   model.add(Dropout(0.25))
#   model.add(Flatten())
#   model.add(Dense(64, activation='relu'))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.25))
#   model.add(Dense(32, activation='relu'))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.25))
#   model.add(Dense(16, activation='relu'))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.25))
#   model.add(Dense(number_of_classes, activation='softmax'))

#   model.compile(loss=loss,
#                 optimizer=keras.optimizers.Adadelta(),
#                 metrics=metrics)

#   model.fit(X_train, np.array(y_train),
#             batch_size=128,
#             epochs=25,
#             verbose=1,
#             validation_data=(xx_val,np.array(yy_val)))


#   score = model.evaluate(xx_test, np.array(yy_test), verbose=1)[1]
#   # prediction = model.predict(X_test)
#   # score = accuracy_score(y_test, prediction)
#   print('Score is', score)
#   return score
  
# data = initilialize_population()
# ga = pyeasyga.GeneticAlgorithm(seed_data=data,
#                                population_size=1,
#                                generations=1,
#                                crossover_probability=0.8,
#                                mutation_probability=0.4,
#                                elitism=True,
#                                maximise_fitness=True)
# ga.create_individual = create_individual
# ga.fitness_function = fitness_function

# start = time.time()
# ga.run()
# end = time.time()
# print('Time taken ------>', end-start)

In [0]:
# best_parameters = ga.best_individual()[1]
# for index,i in enumerate(data):
#   print(i[best_parameters[index]])

# print(ga.best_individual()[0])

# Building CNN Model


In [0]:
# model = Sequential()
# model.add(Conv1D(filters=data[0][best_parameters[0]], kernel_size=data[4][best_parameters[4]], activation='relu', input_shape=input_shape))
# model.add(Conv1D(filters=data[0][best_parameters[0]], kernel_size=data[4][best_parameters[4]], activation='relu', input_shape=input_shape))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=data[1][best_parameters[1]], kernel_size=data[5][best_parameters[5]], activation='relu'))
# model.add(Conv1D(filters=data[1][best_parameters[1]], kernel_size=data[5][best_parameters[5]], activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=data[2][best_parameters[2]], kernel_size=data[5][best_parameters[5]], activation='relu'))
# model.add(Conv1D(filters=data[2][best_parameters[2]], kernel_size=data[5][best_parameters[5]], activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=data[3][best_parameters[3]], kernel_size=data[6][best_parameters[6]], activation='relu'))
# model.add(Conv1D(filters=data[3][best_parameters[3]], kernel_size=data[6][best_parameters[6]], activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# model.add(Dense(32, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# model.add(Dense(16, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# model.add(Dense(number_of_classes, activation='softmax'))
# model.summary()

In [26]:
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(number_of_classes, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 76, 8)             32        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 74, 8)             200       
_________________________________________________________________
batch_normalization_1 (Batch (None, 74, 8)             32        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 37, 8)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 16)            400       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 33, 16)            784       
_________________________________________________________________
batch_normalization_2 (Batch (None, 33, 16)           

In [0]:
# for i in model.layers:
#   name = i.output.name.split('/')[0].split('_')
#   if len(name) > 2:
#     name = name[0] + '_' + name[1]
#   else:
#     name = name[0]
#   print(name, i.output.shape)
#   print('_________________________________________________')

In [28]:
import keras
model.compile(loss=loss,
              optimizer=keras.optimizers.Adadelta(),
              metrics=metrics)

model.fit(X_train, np.array(y_train),
          batch_size=128,
          epochs=100,
          verbose=1,
          validation_data=(xx_val, np.array(yy_val)))

Train on 17110 samples, validate on 12833 samples
Epoch 1/100
17110/17110 [==============================] - 6s 368us/step - loss: 2.4021 - accuracy: 0.1611 - val_loss: 1.8666 - val_accuracy: 0.4304
Epoch 2/100
17110/17110 [==============================] - 4s 256us/step - loss: 1.9566 - accuracy: 0.3487 - val_loss: 1.7802 - val_accuracy: 0.4304
Epoch 3/100
17110/17110 [==============================] - 4s 252us/step - loss: 1.8122 - accuracy: 0.4168 - val_loss: 1.7547 - val_accuracy: 0.4314
Epoch 4/100
17110/17110 [==============================] - 4s 258us/step - loss: 1.7613 - accuracy: 0.4320 - val_loss: 1.7136 - val_accuracy: 0.4356
Epoch 5/100
17110/17110 [==============================] - 4s 257us/step - loss: 1.7313 - accuracy: 0.4396 - val_loss: 1.6894 - val_accuracy: 0.4406
Epoch 6/100
17110/17110 [==============================] - 4s 253us/step - loss: 1.7003 - accuracy: 0.4434 - val_loss: 1.6729 - val_accuracy: 0.4467
Epoch 7/100
17110/17110 [==============================]

In [29]:
score = model.evaluate(xx_test, yy_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

12833/12833 [==============================] - 1s 85us/step
Test loss: 0.7424592813598898
Test accuracy: 0.8222551345825195


In [0]:
if len(np.array(y_test).shape) == 1:
  y_test = [x+1 for x in y_test]
  y_train = [x+1 for x in y_train]

In [31]:
!pip install spectral

     |████████████████████████████████| 194kB 2.6MB/s 


In [0]:
import spectral as sp
prediction = model.predict(X)

if not len(prediction.shape) == 2:
    y_test = [x + 1 for x in y_test]
    y_train = [x + 1 for x in y_train]

if len(prediction.shape) == 2:
    predicted_gt_1 = np.argmax(prediction, axis=1)
    predicted_gt_1_list = list(predicted_gt_1)
    predicted_gt_1_list = [x + 1 for x in predicted_gt_1_list]
else:
    predicted_gt_1_list = prediction

for i in zero_data:
    predicted_gt_1_list.insert(i, 0)

predicted_gt_1_list = np.array(predicted_gt_1_list).reshape(image.shape[0], image.shape[1])
sp.save_rgb('predicted_gt.jpg', predicted_gt_1_list, colors=sp.spy_colors)
